# Session Organiser

## Written by Michael George

Iterate through folders, quickly sorting tracks into desired subfolders, prior to manual checks

In [1]:
import os
import sys
import shutil

from datetime import datetime

corePath = os.path.join('../core')
if corePath not in sys.path:
    sys.path.extend([corePath])

from file_reader import getFileReader

## Main Function

In [2]:
spots = [
    ('Brogborough Lake', (52.043066, -0.587432), (52.050822, -0.566066)),
    ('Portland Harbour', (50.565381, -2.477175), (50.603325, -2.408660)),
    ('West Kirby', (53.363431, -3.192207), (53.372967, -3.180778))
]

def organiseTracks():
    '''Iterate through session archive testing each GPS file'''

    rootDir = os.path.join(projdir, 'sessions', 'unsorted')

    for root, subDirs, files in os.walk(rootDir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            
            if ext in ('.oao', '.sbn', '.sbp'):
                filePath = os.path.join(root, file)
                reader = getFileReader(filePath)
                try:
                    reader.load()
                    
                    if reader.numRecords > 0:

                        meanLat = reader.data['lat'].mean()
                        meanLon = reader.data['lon'].mean()

                        # Determine the spot
                        spotName = 'UNKNOWN'
                        for spot in spots:
                            if spot[1][0] < meanLat < spot[2][0] and spot[1][1] < meanLon < spot[2][1]:
                                spotName = spot[0]
                                break

                        # Rough and ready way to identify the sport
                        maxSpeed = reader.data['sog'].max()
                        if maxSpeed < 5:
                            activity = 'SUPing'
                        elif maxSpeed < 10:
                            activity = 'Wingfoiling'
                        elif maxSpeed < 15:
                            activity = 'Windfoiling'
                        else:
                            activity = 'Windsurfing'

                        # Determine the date
                        dt = datetime.utcfromtimestamp(reader.data['timestamp'].max())
                        sessionYear = dt.strftime('%Y')
                        sessionDate = dt.strftime('%Y-%m-%d')

                        path = os.path.join(projdir, 'sessions', 'sorted', activity, sessionYear, spotName, sessionDate)
                        if not os.path.exists(path):
                            os.makedirs(path)

                        src = os.path.join(root, file)
                        dst = os.path.join(path, file)

                        print('Moving {}...'.format(dst))
                        shutil.move(src, dst)

                except:
                    print('Failed to process {}'.format(filePath))
                    raise

In [3]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    organiseTracks()
    
    print('All done!')

All done!
